In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer,LabelEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from feature_engine.creation import CyclicalFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
# Get the current directory path
current_directory = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_directory, '..', 'src'))

# Add the parent directory to sys.path
parent_dir = os.path.join(grandparent_dir, 'flight_delay')
sys.path.append(parent_dir)

import pre_processing,train_utils,features

In [3]:
#load the data
data_path = '../data/flights_sample_3m.csv'
df_flights = pre_processing.load_data(data_path)

In [4]:
#pre process the data
df_flights = pre_processing.pre_process(df_flights)

In [5]:
numerical_cols = features.numerical_cols
categorical_cols = features.categorical_cols
time_cols = features.time_cols
passthrough_cols = features.passthrough_cols

In [6]:
features_list = numerical_cols + categorical_cols + time_cols + passthrough_cols

In [7]:
df = df_flights[df_flights["covid_data"] == 0]

train_start = "2019-01-01"
train_end   = "2023-01-01"

test_start = "2023-01-01"
test_end   = "2023-09-01"

target_col = "ARR_DELAY"
split_col  = "FL_DATE"

In [8]:
X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df,train_start,train_end,test_start,test_end,split_col,target_col)

In [9]:
pre_processor = train_utils.create_preprocess_pipeline(categorical_cols,numerical_cols,time_cols,passthrough_cols)

In [10]:
model = train_utils.train_model(LGBMRegressor(),pre_processor,X_train,y_train)

/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1900
[LightGBM] [Info] Number of data points in the train set: 1409857, number of used features: 773
[LightGBM] [Info] Start training from score 6.068654


In [11]:
y_test_predictions = model.predict(X_test)

/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [13]:
print(f"""
      MAE : {mean_absolute_error(y_test,y_test_predictions)}
      MAPE : {mean_absolute_percentage_error(y_test,y_test_predictions)}
      """)


      MAE : 26.79095645873963
      MAPE : 648721155328728.9
      
